In [ ]:
!pip install librosa scikit-learn tensorflow


In [ ]:
!pip install soundfile


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [ ]:
def extract_mfcc(file_path):
    # Load the audio file
    audio, sr = librosa.load(file_path, sr=None)
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc = np.mean(mfcc.T, axis=0)  # Taking the mean of MFCC features over time
    return mfcc

def load_data(data_dir):
    features = []
    labels = []

    # Iterate over each folder in the data directory
    for speaker in os.listdir(data_dir):
        speaker_path = os.path.join(data_dir, speaker)
        if os.path.isdir(speaker_path):
            for file in os.listdir(speaker_path):
                file_path = os.path.join(speaker_path, file)
                if file_path.endswith('.wav'):  # Only process .wav files
                    mfcc = extract_mfcc(file_path)
                    features.append(mfcc)
                    labels.append(speaker)

    return np.array(features), np.array(labels)


In [ ]:
# Load the data
data_dir = r'/content/drive/MyDrive/vec/archive/50_speakers_audio_data'  # Replace with your folder path
X, y = load_data(data_dir)

# Encode the labels (speaker names) into numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


<ipython-input-7-f60aff50ae1f>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-7-f60aff50ae1f>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-7-f60aff50ae1f>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/li

In [ ]:
# SVM Classifier
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 96.89%


In [ ]:
##   FOR LARGE DATASETS
# Build a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax')  # Output layer with one neuron per class
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0403 - loss: 18.7438 - val_accuracy: 0.1673 - val_loss: 3.6612
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2398 - loss: 3.2983 - val_accuracy: 0.4202 - val_loss: 2.3929
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4864 - loss: 2.1741 - val_accuracy: 0.6148 - val_loss: 1.6538
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6206 - loss: 1.5633 - val_accuracy: 0.6576 - val_loss: 1.4154
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6935 - loss: 1.2792 - val_accuracy: 0.7743 - val_loss: 1.0786
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7944 - loss: 0.9175 - val_accuracy: 0.8482 - val_loss: 0.8237
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8744 - loss: 0.7063 - val_accuracy: 0.8716 - val_loss: 0.7071
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8780 - loss: 0.6167 - val_accuracy: 0.8911 - val_loss

In [ ]:
def predict_speaker(file_path, model=None):
    mfcc = extract_mfcc(file_path).reshape(1, -1)  # Reshape for prediction
    if model:
        prediction = model.predict(mfcc)
        # Get the class with the highest probability
        predicted_class_index = np.argmax(prediction, axis=1)
        # Inverse transform the predicted class index
        speaker = label_encoder.inverse_transform(predicted_class_index)[0]
    else:
        prediction = svm_classifier.predict(mfcc)
        speaker = label_encoder.inverse_transform([prediction])[0]
    return speaker

# Example usage
test_file = '/content/drive/MyDrive/VoiceRec/VoiceRec/Lab_Based/karuna/Karuna3.wav'  # Replace with your test file path
predicted_speaker = predict_speaker(test_file, model=model)
print(f"Predicted speaker: {predicted_speaker}")


<ipython-input-7-f60aff50ae1f>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Predicted speaker: karuna


In [ ]:
def predict_speaker(file_path, model=None):
    mfcc = extract_mfcc(file_path).reshape(1, -1)  # Reshape for prediction
    if model:
        prediction = model.predict(mfcc)
        # Get the class with the highest probability
        predicted_class_index = np.argmax(prediction, axis=1)
        # Inverse transform the predicted class index
        speaker = label_encoder.inverse_transform(predicted_class_index)[0]
    else:
        prediction = svm_classifier.predict(mfcc)
        speaker = label_encoder.inverse_transform([prediction])[0]
    return speaker

# Example usage
test_file = '/content/drive/MyDrive/VoiceRec/VoiceRec/Lab_Based/swayam/swayam10.wav'  # Replace with your test file path
predicted_speaker = predict_speaker(test_file, model=model)
print(f"Predicted speaker: {predicted_speaker}")


<ipython-input-7-f60aff50ae1f>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted speaker: swayam


In [ ]:
def predict_speaker(file_path, model=None):
    mfcc = extract_mfcc(file_path).reshape(1, -1)  # Reshape for prediction
    if model:
        prediction = model.predict(mfcc)
        # Get the class with the highest probability
        predicted_class_index = np.argmax(prediction, axis=1)
        # Inverse transform the predicted class index
        speaker = label_encoder.inverse_transform(predicted_class_index)[0]
    else:
        prediction = svm_classifier.predict(mfcc)
        speaker = label_encoder.inverse_transform([prediction])[0]
    return speaker

# Example usage
test_file = '/content/drive/MyDrive/VoiceRec/VoiceRec/Lab_Based/anita/Anita1.wav'  # Replace with your test file path
predicted_speaker = predict_speaker(test_file, model=model)
print(f"Predicted speaker: {predicted_speaker}")

<ipython-input-7-f60aff50ae1f>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted speaker: Speaker0032


In [ ]:
def predict_speaker(file_path, model=None):
    mfcc = extract_mfcc(file_path).reshape(1, -1)  # Reshape for prediction
    if model:
        prediction = model.predict(mfcc)
        # Get the class with the highest probability
        predicted_class_index = np.argmax(prediction, axis=1)
        # Inverse transform the predicted class index
        speaker = label_encoder.inverse_transform(predicted_class_index)[0]
    else:
        prediction = svm_classifier.predict(mfcc)
        speaker = label_encoder.inverse_transform([prediction])[0]
    return speaker

# Example usage
test_file = '/content/drive/MyDrive/VoiceRec/VoiceRec/Lab_Based/stefan/5.wav'  # Replace with your test file path
predicted_speaker = predict_speaker(test_file, model=model)
print(f"Predicted speaker: {predicted_speaker}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted speaker: stefan


In [ ]:
def predict_speaker(file_path, model=None):
    mfcc = extract_mfcc(file_path).reshape(1, -1)  # Reshape for prediction
    if model:
        prediction = model.predict(mfcc)
        # Get the class with the highest probability
        predicted_class_index = np.argmax(prediction, axis=1)
        # Inverse transform the predicted class index
        speaker = label_encoder.inverse_transform(predicted_class_index)[0]
    else:
        prediction = svm_classifier.predict(mfcc)
        speaker = label_encoder.inverse_transform([prediction])[0]
    return speaker

# Example usage
test_file = '/content/drive/MyDrive/vec/archive/50_speakers_audio_data/Speaker0028/Speaker28_002.wav'  # Replace with your test file path
predicted_speaker = predict_speaker(test_file, model=model)
print(f"Predicted speaker: {predicted_speaker}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Predicted speaker: Speaker0028
